In [1]:
import torch
from torch.utils.data.dataloader import DataLoader, Dataset
from tqdm.auto import tqdm
import polars as pl
import numpy as np
import random
import os
from sklearn.metrics import mean_squared_error

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)


In [2]:
class EmbeddedFeatures(torch.nn.Module):
    
    def __init__(self, sizes, dims=32):
        super().__init__()
        embs = []
        for c in sizes:
            m = c
            e = torch.nn.Embedding(m, dims)
            embs.append(e)
        self.embeddings = torch.nn.ModuleList(embs)
        
    def forward(self, cats):
        embs = None
        for c, e in zip(cats, self.embeddings):
            if embs is None:
                embs = e(c)
            else:
                embs += e(c)
        embs /= len(cats)
        return embs

class DeepFeatures(torch.nn.Module):
    
    def __init__(self, num_size, embs, depth=3, dims=32):
        super().__init__()
        self.embs = embs
        num_dims = num_size
        #first 
        deep_list = [torch.nn.Linear(dims + num_dims, dims)]
        for _ in range(1, depth):
            deep_list.append(torch.nn.Linear(dims, dims))
        self.deep = torch.nn.ModuleList(deep_list)
        
        
    def forward(self, cats, nums, std=0.5):
        embs = self.embs(cats)
        if nums is not None:
            x = torch.cat((embs, nums), dim=1)
        else:
            x = embs
        if self.training:
            x = x * (1 + std * torch.randn_like(x))
        output = []
        for l in self.deep:
            x = l(x)
            if self.training:
                x = x * (1 + std * torch.randn_like(x))
            output.append(x)
            x = torch.nn.functional.leaky_relu(x)
        return output
    

class DeepMF(torch.nn.Module):
    
    def __init__(self, cat_sizes, num_size, depth=3, dims=32):
        super().__init__()
        embds = EmbeddedFeatures(cat_sizes, dims)
        self.base = DeepFeatures(num_size, embds, depth=depth, dims=32)
        self.click = DeepFeatures(num_size, embds, depth=depth, dims=32)
        self.multi = torch.nn.parameter.Parameter(torch.randn((1,1)))
        self.att = torch.nn.parameter.Parameter(torch.randn((depth, 1)))
        
        
    def forward(self, cats, nums):
        base = self.base(cats, nums)
        click = self.click(cats, nums)
        click_out = None
        for e, (b, c) in enumerate(zip(base, click)):
            c_v = torch.sum(b * c, dim=1, keepdim=True) * self.multi
            if click_out is None:
                click_out = c_v * self.att[e, 0]
            else:
                click_out += (c_v * self.att[e, 0])
        out = click_out
        return torch.nn.functional.sigmoid(out)

In [3]:
def epoch(model, loss_f, optimizer, dl_train, device):
    loss = 0
    model.train()
    for x, y in tqdm(dl_train):
        optimizer.zero_grad()
        cats = [c.to(device) for c in x[:-1]]
        nums = x[-1].to(device)
        y = y.float().to(device)
        y_pred = model(cats, nums)
        c_loss = loss_f(y_pred[:, 0], y)
        c_loss.backward()
        optimizer.step()
        loss += c_loss.cpu().item()
    return loss / len(dl_train)


def predict(model, dl_test, device):
    preds = [] 
    model.eval()
    with torch.no_grad():
        for x, _ in tqdm(dl_test):
            cats = [c.to(device) for c in x[:-1]]
            nums = x[-1].to(device)
            y_pred = model(cats, nums).cpu().numpy()
            preds.append(y_pred[:, 0])
    preds = np.concatenate(preds, axis=0)
    return preds

In [4]:
users = pl.read_csv('ml-100k/u.user', separator='|', has_header=False)
users.columns = ['user', 'age', 'gender', 'occupation', 'zip_code']
movies = pl.read_csv('ml-100k/u.item', separator='|', has_header=False, encoding='utf8-lossy')
movies.columns = ['movie', 'title', 'release_date', 'video release date', \
              'IMDb', 'unknown', 'Action', 'Adventure', 'Animation', \
              'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', \
              'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', \
              'Thriller', 'War', 'Western']
train = pl.read_csv('ml-100k/u1.base', separator='\t', has_header=False)
train.columns = ['user', 'item', 'rating', 'timestamp'] 
test = pl.read_csv('ml-100k/u1.test', separator='\t', has_header=False)
test.columns = ['user', 'item', 'rating', 'timestamp'] 

In [5]:
train = train.join(movies, left_on='item', right_on='movie').join(users, on='user')
test = test.join(movies, left_on='item', right_on='movie').join(users, on='user')

In [6]:
def build_map(ds, column):
    data = ds[column].unique().sort()
    dic = {o: i  for i, o in enumerate(data)}
    return dic

In [7]:
occ_map = build_map(train, 'occupation')
age_map = build_map(train, 'age')

In [8]:
train = train.with_columns(pl.col('occupation').apply(lambda x: occ_map[x]))
train = train.with_columns(pl.col('age').apply(lambda x: age_map[x]))
test = test.with_columns(pl.col('occupation').apply(lambda x: occ_map[x]))
test = test.with_columns(pl.col('age').apply(lambda x: age_map[x]))

In [9]:
class ML100K(Dataset):
    
    def __init__(self, rankings):
        self.users = rankings['user'].to_numpy()
        self.occupation = rankings['occupation'].to_numpy()
        self.age = rankings['age'].to_numpy()
        self.items = rankings['item'].to_numpy()
        self.categories = rankings['unknown', 'Action', 'Adventure', 'Animation', \
              'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', \
              'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', \
              'Thriller', 'War', 'Western'].to_numpy().astype(np.float32)
        self.rankings = (rankings['rating'].to_numpy().astype(np.float32) - 1 ) / 4
    
    def __getitem__(self, idx):
        return (self.users[idx], self.occupation[idx], self.age[idx], self.items[idx], self.categories[idx, :]), self.rankings[idx]
                                                      
    def __len__(self):
        return self.users.shape[0]

In [10]:
dl_train = DataLoader(ML100K(train), \
                      batch_size=64, shuffle=True)
dl_test = DataLoader(ML100K(test), \
                      batch_size=len(test), shuffle=False)

In [11]:
device = 'cuda'

model = DeepMF([train['user'].max() + 1,\
                train['occupation'].max() + 1,\
                train['age'].max() + 1,\
                train['item'].max() + 1], 19).to(device)
loss_f = torch.nn.MSELoss() #torch.nn.BCELoss()
optimizer = torch.optim.RAdam(model.parameters())

In [12]:
if not os.path.exists('ml100k-exp-full-batch.pt'):
    for i in range(40):
        l = epoch(model, loss_f, optimizer, dl_train, device)
        print(f'{i}: Current loss in training {l}')
        y_pred = predict(model, dl_test, device)
        print(f'RMSE Score: {mean_squared_error(test["rating"].to_numpy(), y_pred * 4 + 1, squared=False)}')
    torch.save(model.state_dict(), 'ml100k-exp-full-batch.pt')
else:
    model.load_state_dict(torch.load('ml100k-exp-full-batch.pt'))


y_pred = predict(model, dl_test, device)

print(f'RMSE Score: {mean_squared_error(test["rating"].to_numpy(), y_pred * 4 + 1, squared=False)}')

  0%|          | 0/1250 [00:00<?, ?it/s]

0: Current loss in training 0.0777442572504282


  0%|          | 0/1 [00:00<?, ?it/s]

RMSE Score: 1.1096537064303265


  0%|          | 0/1250 [00:00<?, ?it/s]

...

39: Current loss in training 0.045938333784043786


  0%|          | 0/1 [00:00<?, ?it/s]

RMSE Score: 0.9362451687091167


  0%|          | 0/1 [00:00<?, ?it/s]

RMSE Score: 0.9362451687091167
